In [5]:
import pandas as pd
import sqlite3

In [55]:
connection = sqlite3.connect('database.db')

In [63]:
database = '''
CREATE TABLE table1 (

ID INTEGER PRIMARY KEY, 
Vehicle_type TEXT,
Milleage INTEGER, 
Max_speed INTEGER, 
Passenger_count INTEGER, 
Category NOT NULL CHECK (category IN ('A', 'B', 'C'))

)'''




In [69]:
cursor = connection.cursor()
try:
    cursor.execute(database)
    connection.commit()

except Exception as e:
    print(e)
    connection.rollback()

In [81]:
select_query = '''

SELECT *
FROM table1
'''
gg = cursor.execute(select_query)
columns = [col[0] for col in gg.description]
result = gg.fetchall()
print(columns)
print(result)

['ID', 'Vehicle_type', 'Milleage', 'Max_speed', 'Passenger_count', 'Category']
[]


In [85]:
df = pd.read_sql_query(select_query, connection)
df

,ID,Vehicle_type,Milleage,Max_speed,Passenger_count,Category


In [239]:
import random

Category = ['A']*3 + ['B']*12 + ['C']*5

random.shuffle(Category)

data_table1 = []

#chat gpt запропонував мені додати це, аби сюди записувати результати, але я не можу цього зрзуміти. навіщо воно нам? 

for cat in Category:
    if cat == 'A':
        Vehicle_type = 'moto'
    elif cat == 'B':
        Vehicle_type = 'SUV'
    elif cat == 'C':
        Vehicle_type = 'truck'
        
    Milleage = random.randint(10000, 100000)
    Max_speed = random.randint(100, 200)
    Passenger_count = random.randint(1, 5) if cat == 'C' else random.randint(2, 3)
    
    data_table1.append((Vehicle_type, Milleage, Max_speed, Passenger_count, cat))

cursor.executemany('''
INSERT INTO Table1 (Vehicle_type, Milleage, Max_speed, Passenger_count, Category)
VALUES (?, ?, ?, ?, ?)
''', data_table1)

#перша таблиця заповнена

In [327]:
#початок заповнення другої таблиці

cursor.execute("SELECT ID, Category FROM Table1")
vehicle_data = cursor.fetchall()

#це підказав chat gpt, я не знав, для чого f. 
Driver_name = [f"Driver_{i}" for i in range(1, 21)]

Driver_categories = []

for i, (vehicle_id, category) in enumerate(vehicle_data):
    
    if i < 13:
        if i < 3:
            driver_category = 'A'
        elif i < 15:
            driver_category = 'B'
        else:
            driver_category = 'C'
    else:
        
        driver_category = random.choice(['A', 'B', 'C'])
        while driver_category == category:
            driver_category = random.choice(['A', 'B', 'C'])
            
    Driver_categories.append(driver_category)

dataframe = pd.DataFrame(list(zip(Driver_name, Driver_categories)), 
columns = ['Driver name', 'Driver category'])

dataframe.to_sql('table2', connection, if_exists='replace', index = False)
connection.commit()

,Driver name,Driver category
0,Driver_1,A
1,Driver_2,A
2,Driver_3,A
3,Driver_4,B
4,Driver_5,B
5,Driver_6,B
6,Driver_7,B
7,Driver_8,B
8,Driver_9,B
9,Driver_10,B


In [467]:
#table 3


cursor.execute('''
CREATE TABLE IF NOT EXISTS Table3 (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Category TEXT NOT NULL,
    Permitted_roads TEXT NOT NULL
)
''')


roads = ['CR22', 'AT12', 'BR34', 'DT56', 'EF78', 'GH90', 'IJ12', 'KL34', 'MN56', 'OP78']
categories = ['A', 'B', 'C']

data_table3 = []

for road in roads:
    category = random.choice(categories)
    data_table3.append((category, road))

dataframe2 = pd.DataFrame(data_table3, columns = ['Categories','Permitted_roads'])

dataframe2

cursor.executemany('''
INSERT INTO Table3 (categories, roads)
VALUES (?, ?)
''', data_table3)

connection.commit()

In [551]:
select = '''

SELECT *
FROM Table2, Table1
'''

#не знаю, як тут правильно прописати, аби нам дало правильні таблиці.
#я розумію, що потрібно зробити, але я не можу написати код до цього, бо він видає помилки. гугл, stackoverflowта chat gpt не допомогли. 


df_correct_matching = pd.read_sql(select, connection)

DatabaseError: Execution failed on sql '

SELECT *
FROM Table2, Table1
WHERE Driver category = 


': near "category": syntax error

In [559]:
#середня швидкість транспортних засобів по категоріям

last_query = '''

SELECT Category, avg(Max_speed) AS average_speed
FROM Table1
GROUP BY category;

'''

pd.read_sql_query(last_query, connection)

,Category,average_speed
0,A,154.000000
1,B,152.208333
2,C,148.900000


In [565]:
#максимальний пробіг по категоріям

last_query2 = '''

SELECT Category, max(Milleage) AS max_milleage
FROM Table1
GROUP BY category;

'''

pd.read_sql_query(last_query2, connection)



,Category,max_milleage
0,A,61765
1,B,97595
2,C,99749


In [573]:
#максимальна швидкість транспортного засобу з кількістю пасажирів більше 8


last_query3 = '''

SELECT Category, max(Max_speed) AS max_speed8
FROM Table1
WHERE Passenger_count > 8 
GROUP BY category;

'''

pd.read_sql_query(last_query3, connection)

,Category,max_speed8
